# Knowledge and Data 2020: Practical Assignment 2
## Manipulate local and external RDF Knowledge Graphs 

YOUR NAME: Seeun Park

YOUR VUNetID: spk760

*(If you do not provide your name and VUNetID we will not accept your submission).*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of RDF Data using the rdflib library. You should be able to: 

1. Add and retrieve information from a local RDF database
2. Represent RDF data in other formats, such as the .dot format for graph visualisation
3. Retrieve information from an RDF database created from Web Data
4. Query information from the Web with SPARQL

### Practicalities

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everything is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook, i.e. with an .ipynb extension. Please use as name of the 
Notebook your studentID+Assignment2.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

# A. Tasks related to local RDF Knowledge Graphs

This first cell will open a file 'example-from-slide.ttl' using the rdflib library. The first Practical Assignment should have taught you that manipulating symbols as strings is a major pain. 

Programming libraries, such as **rdflib**, help you with this mess once and for all, by parsing the files, creating appropriate datastructures (Graph()) and providing useful functions (such as serialize(), save() and much more). 
Check the website of rdflib http://rdflib.readthedocs.io/: this library does most of the hard work for you.

Before starting with the tasks of this assignment, do not forget to install **rdflib** so we can start using it. Installing libraries in Python is very simple. Just open your terminal and write the following command:

    $ pip install rdflib

For more details on how to install pip and Python libraries, you can check the [preliminaries document](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/preliminaries.md).  

In [1]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef, RDFS

g = Graph()

EX = Namespace('http://example.com/kad2020/')
g.bind('ex',EX)

def serialize_graph():
    # g.serialize() returns a string
    print(g.serialize(format='turtle'))

def save_graph(filename):
    with open(filename, 'w') as f:
        g.serialize(f, format='nt')
        
def load_graph(filename):
    with open(filename, 'r') as f:
        g.parse(f, format='turtle')   

The file 'example-from-slides.ttl' formalises the knowledge base from the slides from Module 1, and a bit more. 

Here is how it looks when you load it into your program and serialise it with rdflib in turtle. 

In [2]:
load_graph('example-from-slides.ttl')
serialize_graph()

@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Germany a ex:EuropeanCountry .

ex:Netherlands a ex:Country ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:Amsterdam a ex:Capital .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City .

ex:Capital rdfs:subClassOf ex:City .




Now, we can manipulate the graph very easily, e.g. like in the following very simple function, which returns the predicate(s) that relate a subject to a literal object: 

In [3]:
for s,p,o in g:
    if type(o) is Literal:
        print(p)

http://example.com/kad2020/has_Name


### - Task 1: (1 Point) Add information to an RDF graph

Add 10 triples to the knowledge graph. Make sure that they have the right namespaces. 

Similarily to the triples already present in the file 'example-from-slides.ttl', add at least:
- a new country with its name and capital 
- one triple with a new predicate

Check: http://rdflib.readthedocs.io/en/stable/intro_to_creating_rdf.html

In [4]:
ex = Namespace("http://example.com/kad2020/")
owl = Namespace('http://www.w3.org/2002/07/owl#')

# add 10 triples here to the graph 'g' (do not forget the namespaces).
g.add((ex.Korea, RDF.type, ex.Country))
g.add((ex.Korea, ex.has_Name, Literal("Korea")))
g.add((ex.Korea, ex.neighbours, ex.China))
g.add((ex.Korea, ex.neighbours, ex.Japan))
g.add((ex.Korea, ex.has_Capital, ex.Seoul))
g.add((ex.Seoul, ex.has_Name, Literal("Seoul")))
g.add((ex.Korea, RDF.type, ex.AsianCountry))
g.add((ex.Asia, ex.has_Name, Literal("Asia")))

#new predicates
g.add((ex.Korea, ex.in_continent, ex.Asia))
g.add((ex.Korea, ex.airport_at, ex.Incheon))

serialize_graph()

@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Germany a ex:EuropeanCountry .

ex:Korea a ex:AsianCountry,
        ex:Country ;
    ex:airport_at ex:Incheon ;
    ex:has_Capital ex:Seoul ;
    ex:has_Name "Korea" ;
    ex:in_continent ex:Asia ;
    ex:neighbours ex:China,
        ex:Japan .

ex:Netherlands a ex:Country ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:Amsterdam a ex:Capital .

ex:Asia ex:has_Name "Asia" .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Seoul ex:has_Name "Seoul" .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City .

ex:Capital rdfs:subClassOf ex:City .




*After you ran the previous code (adding triples) the next cells will be executed on your extended graph. That is ok.*

### - Task 2a: (1 Point) Get structured information from an RDF graph (all Literals)

Use the functions available in the RDFLib library. Write a small function to print all Literals. 

Hint: there is a function in rdflib to test the type of an object (check previous examples in this notebook)

In [5]:
for s,p,o in g:
    if type(o) is Literal:
        print(o)
    pass

Seoul
Asia
Korea
The Netherlands


### - Task 2b: (1 Point) Get structured information from an RDF graph (all unique Predicates)

Please provide another function that gives a **unique** list of the predicates, ordered by occurrence (most occurring first). The answer will look like this: 
<br>http://www.w3.org/2000/01/rdf-schema#label
<br>http://www.w3.org/1999/02/22-rdf-syntax-ns#type
<br>http://example.com/sw2016/locatedIn
<br>http://www.w3.org/2000/01/rdf-schema#range

In [6]:
predicate = []
predicate_list = {}
for s,p,o in g:
    if p in predicate_list:
        predicate_list[p] += 1
    else:
        predicate_list[p] = 1
        predicate.append(p)
sort = sorted(predicate_list.items(), key = lambda x: x[1], reverse = True)
for i in sort:
    print(i[0])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://example.com/kad2020/has_Name
http://example.com/kad2020/neighbours
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://example.com/kad2020/has_Capital
http://www.w3.org/2000/01/rdf-schema#range
http://www.w3.org/2000/01/rdf-schema#subPropertyOf
http://example.com/kad2020/airport_at
http://example.com/kad2020/in_continent
http://www.w3.org/2000/01/rdf-schema#domain


# B. Tasks related to Graph visualisations 

### - Task 3a: (1 Point) From RDF to .dot 


In the lecture, we have seen two ways of writing a knowledge graph (simple n-triples, and simple turtle). Let us consider a 3rd syntax, this time a syntax that is useful for visualisation. One standard for visualising graphs is the .dot format.

Print the knowledge graph in .dot file format. Check https://graphviz.gitlab.io/documentation/ for the documentation. You will only need very little of this information, and the most relevant information can be found in the examples that are given. 

<br>Basically, an RDF graph in .dot format starts with 
<br>digraph G { 
    and then a list of links of the following form 
<br>s -> o [label="p"]
    for every (s p o ) in KG (separated by ;
<br>Do not forget to end with a closing bracket. }

An example is 
     
     digraph G { s1 -> o1 [label="p1"] ; s2 -> o2 [label="p2"] } 
     
for an RDF graph {(s1 p1 o1),(s2 p2 o2)}

*You can check how your graph looks like. Just copy & paste your output into: http://www.webgraphviz.com/*

In [7]:
print("digraph G {")

for s,p,o in g:
    if type(o) == Literal:
        o = '"' + o + '"'
    new_s = s.split("/")[-1]
    new_p = p.split("/")[-1].split("#")[-1]
    new_o = o.split("/")[-1]
    print(f'{new_s} -> {new_o} [label = "{new_p}"]')

print("}")    

digraph G {
Seoul -> "Seoul" [label = "has_Name"]
hasCapital -> containsCity [label = "subPropertyOf"]
Netherlands -> Country [label = "type"]
Korea -> Country [label = "type"]
Korea -> Incheon [label = "airport_at"]
Korea -> AsianCountry [label = "type"]
Korea -> China [label = "neighbours"]
EuropeanCountry -> Country [label = "subClassOf"]
Germany -> EuropeanCountry [label = "type"]
Asia -> "Asia" [label = "has_Name"]
Amsterdam -> Capital [label = "type"]
Netherlands -> Belgium [label = "neighbours"]
Netherlands -> Amsterdam [label = "has_Capital"]
Belgium -> Country [label = "type"]
Korea -> "Korea" [label = "has_Name"]
Capital -> City [label = "subClassOf"]
Korea -> Asia [label = "in_continent"]
containsCity -> City [label = "range"]
Netherlands -> "The Netherlands" [label = "has_Name"]
hasCapital -> Capital [label = "range"]
Korea -> Seoul [label = "has_Capital"]
containsCity -> Country [label = "domain"]
Korea -> Japan [label = "neighbours"]
}


### - Task 3b: (1 Point) From RDF to .dot with "semantic information"

There is a conceptual distinction between properties, instances and classes (sets of instances). A simple way of checking is the following

1. in a triple (s a o), with predicate a (which is a special abbreviation for the predicate rdf:type), the s is an Instance, and o is a Class. 
2. in a triple (s rdfs:subClassOf o) both s and o are Classes. 
3. in a triple (p rdfs:domain o) p is a Property and o is a Class. 
4. in a triple (p rdfs:range o)  p is a Property and o is a Class. 

Make a .dot representation for an RDF graph that distinguishes between types of links (RDF vocabulary vs others) and types of nodes (Classes versus Instances) via different colors. 

*You can check how your graph looks like. Just copy & paste your output into: http://www.webgraphviz.com/*

In [8]:

print("digraph G {")

yellow = '[color = "Yellow"]'
green = '[color = "Green"]'
purple = '[color = "Purple"]'

for s,p,o in g:
    if type(o) == Literal:
        o = '"' + o + '"'
        
    new_s = s.split('/')[-1]
    new_p = p.split('/')[-1].split('#')[-1]
    new_o = o.split('/')[-1]
    
    if p == RDFS.subClassOf:
        print(new_s + " " + yellow)
        print(new_o + " " + green)
    
    elif p == RDF.type:
        print(new_s + " " + yellow)
        print(new_o + " " + green)
        
    elif p == RDFS.range:
        print(new_s + " " + purple)
        print(new_o + " " + green)
    
    elif p == RDFS.domain:
        print(new_s + " " + purple)
        print(new_o + " " + green)
        
    print(f'{new_s} -> {new_o} [label="{new_p}"]')
print("}")

digraph G {
Seoul -> "Seoul" [label="has_Name"]
hasCapital -> containsCity [label="subPropertyOf"]
Netherlands [color = "Yellow"]
Country [color = "Green"]
Netherlands -> Country [label="type"]
Korea [color = "Yellow"]
Country [color = "Green"]
Korea -> Country [label="type"]
Korea -> Incheon [label="airport_at"]
Korea [color = "Yellow"]
AsianCountry [color = "Green"]
Korea -> AsianCountry [label="type"]
Korea -> China [label="neighbours"]
EuropeanCountry [color = "Yellow"]
Country [color = "Green"]
EuropeanCountry -> Country [label="subClassOf"]
Germany [color = "Yellow"]
EuropeanCountry [color = "Green"]
Germany -> EuropeanCountry [label="type"]
Asia -> "Asia" [label="has_Name"]
Amsterdam [color = "Yellow"]
Capital [color = "Green"]
Amsterdam -> Capital [label="type"]
Netherlands -> Belgium [label="neighbours"]
Netherlands -> Amsterdam [label="has_Capital"]
Belgium [color = "Yellow"]
Country [color = "Green"]
Belgium -> Country [label="type"]
Korea -> "Korea" [label="has_Name"]
Capit

### - Task 4a: (1 Point) Deriving implicit knowledge (a bit of schema)

We will look into Schema information in the latter modules, but let us try already to find some implicit information in a first bit of inferencing: whenever there are two statements (s a o) and (o rdfs:subClassOf o2) we can derive (and later prove) that (s a o2). 

Write a procedure that adds all implied triples to our knowledge graph. 

In [15]:
main_list = []
sub_list = []

for s,p,o in g:
    new_s = s.split('/')[-1]
    new_p = p.split('/')[-1].split('#')[-1]
    new_o = o.split('/')[-1]
    
    if p == RDF.type:
        main_list.append([s,p,o])
    elif p == RDFS.subClassOf:
        sub_list.append([s,p,o])

print(main_list)
print(sub_list)

[[rdflib.term.URIRef('http://example.com/kad2020/Netherlands'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad2020/Country')], [rdflib.term.URIRef('http://example.com/kad2020/Korea'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad2020/Country')], [rdflib.term.URIRef('http://example.com/kad2020/Korea'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad2020/AsianCountry')], [rdflib.term.URIRef('http://example.com/kad2020/Germany'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad2020/EuropeanCountry')], [rdflib.term.URIRef('http://example.com/kad2020/Amsterdam'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad2020/Capital')], [rdflib.term.URIRef('http://example.com

### - Task 4b: (Optional - 1 Extra Point) Visualising implicit knowledge

Produce a .dot version of the graph with those implies implicit triples, and mark the edges of those triples with a different color or arrow style. 

In [ ]:
# Your code here

# C. Tasks related to local copies of external RDF Datasets using SPARQL

Until now, we have manipulated local knowledge graphs, but as we claimed in the lectures, the advantage of knowledge graphs is that they can easily be linked with other datasets on the Web. 

In the remaining 3 tasks, we will manipulate data from the Web, and ask complex queries over this web data. 

In the first task, we will access web data, make a local copy of it, and then query it. In the other two tasks, we will query live data directly from web Knowledge Graphs (in this case, the SPARQL endpoint of DBPedia). 

### - Task 5: (1 Point) Show and manipulate data about RDF resources on the Web 

With rdflib we can easily load a local graph, but we can just as well retrieve a graph from the Web. Here, we will do so using the *requests* library, which allows us to fire a request to any server and/or SPARQL endpoint and to capture the response. The following snippet does so for the resource Amsterdam Dbpedia, by using the 'DESCRIBE' keyword to give us all triples about Amsterdam, and then loads it in a RDFlib Graph object.

In [ ]:
# install the library
%pip install requests

In [ ]:
import requests

endpoint = "https://dbpedia.org/sparql"
query = 'DESCRIBE <http://dbpedia.org/resource/Amsterdam>'

payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g = Graph()
g.parse(data=response.text, format='ttl')

Now do the same for Rotterdam

In [ ]:
query = 'DESCRIBE <http://dbpedia.org/resource/Rotterdam>'

payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g.parse(data=response.text, format='ttl')  # calling parse again merges the graphs

Let us start by showing diverse bits of information w.r.t  Amsterdam and Rotterdam in DBPedia. It is very similar to task 1, but now with Web Data. 

First, query the graph g (now containing the DBPedia information about Amsterdam and Rotterdam) and check whether you can find someone who was born in Amsterdam (is dbo:birthPlace of) and died in Rotterdam (is dbo:deathPlace of)?

In [ ]:
qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
        WHERE {
            ?s dbo:birthPlace dbr:Amsterdam .
            ?s dbo:deathPlace dbr:Rotterdam .
        }
        LIMIT 10
       """)
for row in qres:
    print("%s" % row)

Write a query to check whether there is an album that was recorded both in Rotterdam and Amsterdam? You need to look at the data to know which property you should check for. 

To get an intuition of what is in the knowledge graph you might want to look at the human readable rendering on : http://dbpedia.org/resource/Amsterdam

In [ ]:
qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
        WHERE {
            ?s dbo:recordedIn dbr:Amsterdam .
            ?s dbo:recordedIn dbr:Rotterdam .
        }
        LIMIT 10
       """)
for row in qres:
    print("%s" % row)

### - Task 6: (2 Points) Ask SPARQL against live data using Yasgui

Yasgui (http://yasgui.org/) is a nice graphical interface for asking queries.

Run a new query against http://dbpedia.org/sparql that does the following:

- Find all languages spoken in countries that are not official languages of that country
- The query should return two colums: the country, and the number of languages.
- Order the countries by the number of unofficial languages, from high to low.

In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
SELECT ?country (COUNT(?spokenIn) AS ?numLanguages)
     WHERE {
         ?country rdf:type dbo:Country .
         {?spokenIn dbo:spokenIn ?country .}
         MINUS{?country dbo:officialLanguage ?officialLanguage .}
}ORDER BY DESC(?numLanguages)

### - Task 7: (1 Point) SPARQL 

Write a SPARQL query which returns all relationship(s) between the series "The Office (UK)" and the actor "Ricky Gervais" (literally).

Be careful to check for relations in both directions (but not necessarily the same relation in both directions).  

Use Yasgui to design the correct SPARQL query, and copy paste it in the cell below. 

In [ ]:
# Add here the SPARQL query (not Python) code. (copy & paste from Yasgui)
# When executing against Yasgui you should get an answer. 
# Don't worry that executing this cell will return an error message (a SPARQL query is not a Python code, so it should give an error message here).

In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
SELECT DISTINCT ?p
	WHERE {
  {dbr:Ricky_Gervais ?p dbr:The_Office_\(UK_TV_series\)}
  {dbr:The_Office_\(UK_TV_series\) ?p dbr:Ricky_Gervais}
	}